In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Calculate limb darkening coefficients

First... some imports!

In [34]:
from ExoCTK.ExoCTK import core
from ExoCTK.ExoCTK.limb_dark import ldcfit as lf
from ExoCTK.ExoCTK.limb_dark import ldcplot as lp

To calculate the limb darkening coefficients, we need a model grid. 

I'm going to use the Phoenix ACES models for this example but any grid can be loaded into a `core.ModelGrid()` object if the spectra are stored as FITS files.

We will pass a directory+wildcard path to the initialize the model grid since the parameters are included in the filenames.

In [27]:
fits_files = '../../../limb_dark_jeff/limb/specint/lte*.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits'
grid = core.ModelGrid(fits_files)
print(grid.data)

 Teff  logg ...                          filename                         
------ ---- ... ----------------------------------------------------------
2300.0  4.0 ... lte02300-4.00+0.5.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2300.0  4.0 ... lte02300-4.00-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2300.0  4.0 ... lte02300-4.00-0.5.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2300.0  4.5 ... lte02300-4.50+0.5.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2300.0  4.5 ... lte02300-4.50-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2300.0  4.5 ... lte02300-4.50-0.5.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2300.0  5.0 ... lte02300-5.00+0.5.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2300.0  5.0 ... lte02300-5.00-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2300.0  5.0 ... lte02300-5.00-0.5.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2300.0  5.5 ... lte02300-5.50+0.5.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
   ...  ... ...                                                        ...
2800.0  4.5 ... lte02800-

Now let's customize it to our desired effective temperature, surface gravity, metallicity, and wavelength ranges by running the `customize()` method on our grid.

In [36]:
grid.customize(teff_range=(2400,2800), logg_range=(4,5.5), FeH_range=(0,0), wavelength_range=(1.1,1.6))

20/20 spectra in parameter range Teff: (2400, 2800) , logg: (4, 5.5) , FeH: (0, 0) , wavelength: (1.1, 1.6)
 Teff  logg ...                          filename                         
------ ---- ... ----------------------------------------------------------
2400.0  4.0 ... lte02400-4.00-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2400.0  4.5 ... lte02400-4.50-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2400.0  5.0 ... lte02400-5.00-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2400.0  5.5 ... lte02400-5.50-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2500.0  4.0 ... lte02500-4.00-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2500.0  4.5 ... lte02500-4.50-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2500.0  5.0 ... lte02500-5.00-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2500.0  5.5 ... lte02500-5.50-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2600.0  4.0 ... lte02600-4.00-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits
2600.0  4.5 ... lte02600-4.50-0.0.PHOENIX-ACES-AGSS-COND-SPECINT-20

Now we can caluclate the limb darkening coefficients of a single model on the grid...

In [30]:
teff, logg, FeH = 2500, 5.0, 0.0
ldc_grid_point = lf.ldc(teff, logg, FeH, grid, 2)

...or a single model off the grid...

In [31]:
teff, logg, FeH = 2532, 5.1, 0.0
ldc_off_grid_point = lf.ldc(teff, logg, FeH, grid, 2)

Teff: 2532  logg: 5.1  FeH: 0.0  model not in grid. Calculating...


/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:2405: RuntimeWarning: invalid value encountered in true_divide
  (grid[i + 1] - grid[i]))


...or the entire grid!

In [33]:
whole_grid = lf.ldc_grid(grid, 2)

Enjoy!